<h1 align="center">Защита данных клиентов</h1>

## Краткое описание проекта

В рамках данного проекта нужно защитить данные клиентов страховой компании «Хоть потоп». **Целью** проекта является разработка такого метода преобразования данных, чтобы по ним было сложно восстановить персональную информацию. При этом подбирать наилучшую модель не нужно. Для выполнения цели потребуется решить следующие *задачи*:

1. Получить данные;
2. Выяснить, меняется ли качество модели при умножении признаков на обратимую матрицу;
3. Разработать метод преобразования данных, обосновать корректность его работы;
4. Проверить метод на предоставленных данных;

## Загрузка данных

Импортируем необходимые библиотеки, прочитаем предоставленный файл и сохраним его в переменной `insurance`.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import os
from pathlib import Path
import urllib

In [2]:
Path('/datasets').mkdir(parents=True,exist_ok=True)
def get_file(file_name, url):
    
    if not os.path.exists(file_name):
        print(file_name,'не найден. Файл будет загружен из сети')
        _ = urllib.request.urlretrieve(url, file_name)

urls = {
    'insurance': ('/datasets/insurance.csv', 'https://.../insurance.csv')   
}

[get_file(*urls[k]) for k in urls]

[None]

In [3]:
insurance = pd.read_csv('/datasets/insurance.csv')

Ознакомимся с данными.

In [4]:
insurance.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
insurance.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [7]:
insurance.duplicated().sum()

153

В данных нет пропусков, отрицательных значений возраста, зарплаты и количества членов семьи, однако есть дубликаты. В рамках данного проекта не будем выполнять предобработку данных (переименование столбцов, удаление дубликатов, изменение типов данных), т.к. не стоит такой задачи.

Сохраним в переменных `X` и `y` матрицу признаков и вектор целевого признака соответственно.

In [8]:
X = insurance.drop('Страховые выплаты', axis=1).values
y = insurance['Страховые выплаты'].values

### Вывод

В этом разделе мы получили данные, ознакомились с ними и сохранили признаки и целевой признак в отдельных переменных.

## Умножение матриц

В этом разделе необходимо выяснить, изменится ли качество линейной регрессии при умножении признаков на обратимую матрицу. Сначала обратимся к теории.

### Теория

Задача обучения линейной регрессии:

$$
w = \arg\min_w MSE(Xw, y)
$$
где:
- $X$ — матрица признаков (нулевой столбец состоит из единиц);
- $y$ — вектор целевого признака;
- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу).

Предсказания определяются по формуле:

$$
a = Xw
$$



Оптимальные веса определяются по формуле:

$$
w = (X^T X)^{-1} X^T y
$$


которая в свою очередь получена из уравнения линейной регрессии:

$$
y = Xw
$$

При умножении матрицы признаков на матрицу $P$ уравнение линейной регрессии запишется в виде:

$$
y = XPw_{new}
$$
где:
- $P$ — матрица, на которую умножаются признаки (обратимая);
- $w_{new}$ — новый вектор весов линейной регрессии.

Поскольку вектор целевого признака не меняется в процессе обучения, приравняем правые части уравнений и выполним ряд преобразований:

$$
Xw = XPw_{new}
$$

Мы не можем просто домножить обе части уравнения на обратную матрицу $X^{-1}$, поскольку ее может просто не существовать для  матрицы признаков (как в нашем случае, наша матрица признаков не квадратная). Поэтому домножим обе части уравнения на транспонированную матрицу признаков $X^T$:

$$
X^T Xw = X^T XPw_{new}
$$

Исходя из свойств умножения матриц $X^T X$ - квадратная, а значит может иметь обратную. Домножим обе части равнения на $(X^T X)^{-1}$:

$$
(X^T X)^{-1} X^T Xw =(X^T X)^{-1} X^T XPw_{new}
$$

$$
Ew =EPw_{new}
$$

$$
w =Pw_{new}
$$

$$
w_{new} = P^{-1}w
$$


Видно, что новый вектор весов линейной регрессии есть скалярное произведение обратной матрицы, на которую умножались признаки, и изначального вектора весов.

Посмотрим, как поменяются предсказания модели при умножении признаков на матрицу $P$.

$$
a = X P w_{new} = X P P^{-1}w = X E w = Xw
$$

В теории предсказания модели не изменятся, а значит не изменится и качество модели. Проверим это на практике.

### Практика

Для обучения моделей создадим класс `LinearRegression`, который будет в том числе включать в себя метод *score*, возвращающий метрику **R2** обученной модели.

In [9]:
class LinearR:
    def __init__(self, features, target):
        self.model = LinearRegression()
        self.preds = self.model.fit(features, target).predict(features)
        self.score = r2_score(target, self.preds)

Получим метрику **R2** для неизмененных данных.

In [10]:
original = LinearR(X, y)
original.score

0.42494550286668

**R2** линейной регрессии, обученной на неизмененных данных, **~0.425**.

Создадим случайную квадратную матрицу нужного размера и проверим, обратима ли она.

In [11]:
P = np.random.randint(100, size=(X.shape[1], X.shape[1]))
P

array([[35,  5, 83, 67],
       [83, 76,  0, 43],
       [55, 83, 45, 18],
       [11, 35, 85, 54]])

In [12]:
np.linalg.inv(P)

array([[ 0.02367551, -0.00404487,  0.01548268, -0.03131516],
       [-0.01981317,  0.00634514, -0.00093545,  0.01984222],
       [ 0.01187981, -0.014699  ,  0.01631706, -0.00847402],
       [-0.01068061,  0.01984868, -0.02823183,  0.02537557]])

Случайная матрица **P** обратима. Умножим на нее матрицу признаков **X**, обучим модель на измененных данных и получим метрику **R2**.

In [13]:
X_altered = X @ P
altered = LinearR(X_altered, y)
altered.score

0.424945502866681

Видно, что метрика **R2** не изменилась. Посмотрим на вектроры весов.

Получим $P^{-1}w$.

In [14]:
np.linalg.inv(P) @ original.model.coef_

array([ 0.00046808, -0.00019967, -0.00031575,  0.00027983])

Теперь получим $w$.

In [15]:
altered.model.coef_

array([ 0.00046808, -0.00019967, -0.00031575,  0.00027983])

На практике равенство $w_{new} = P^{-1}w$ выполняется.

### Вывод

В данном разделе выяснили в теории и убедились на практике, что умножение признаков на случайную обратимую матрицу не влияет на качество линейной регрессии. Это станет идеей для разработки агоритма преобразования данных пользователей.

## Алгоритм преобразования

Осуществим преобразование данных следующим образом:
- Выделим признаки;
- Преобразуем их в матрицу векторов $X$;
- Получим измененную матрицу признаков $A$, умножив матрицу признаков $X$ на случайную обратимую матрицу $P$.

На преобразованных данных можно обучать модель линейной регрессии. Ключом к исходным данным будет матрица $P$. Получить исходные данные можно умножением $A$ на $P^{-1}$.

Создадим класс `Cipher`, методами которого будут:
- *cipher* - получение зашифрованных данных в виде матрицы;
- *decipher* - восстановление данных в виде датафрейма.
На вход наш класс будет принимать признаки, преобразовывать их в матрицу векторов внутри себя. Хранить матрицу-ключ будет хранить в приватной переменной.

In [16]:
class Cipher:
    def __init__(self, features):
        self.X = features.values
        self.columns = features.columns
    
    def __make_matrix(self, matrix):#создаем случайную матрицу
        self.__P = np.random.randint(1000, size=(matrix.shape[1], matrix.shape[1]))
        try:#проверяем, обратима ли она
            np.linalg.inv(self.__P)
        except np.linalg.LinAlgError:#если нет, создаем заново
            self.__make_matrix(X)
        return self.__P
  
    def cipher(self):
        return self.X @ self.__make_matrix(self.X)
    
    def decipher(self, matrix):
        self.__decipher = matrix @ np.linalg.inv(self.__P)
        return pd.DataFrame(matrix @ np.linalg.inv(self.__P), columns=self.columns)


## Проверка алгоритма

Проверим работу нашего алгоритма.

In [17]:
cip = Cipher(insurance.drop('Страховые выплаты', axis=1))

In [18]:
cip.X

array([[1.00e+00, 4.10e+01, 4.96e+04, 1.00e+00],
       [0.00e+00, 4.60e+01, 3.80e+04, 1.00e+00],
       [0.00e+00, 2.90e+01, 2.10e+04, 0.00e+00],
       ...,
       [0.00e+00, 2.00e+01, 3.39e+04, 2.00e+00],
       [1.00e+00, 2.20e+01, 3.27e+04, 3.00e+00],
       [1.00e+00, 2.80e+01, 4.06e+04, 1.00e+00]])

Преобразуем признаки и посмотрим на них.

In [19]:
cip_vector = cip.cipher()
cip_vector

array([[13067872., 37401269.,  8461110., 36912094.],
       [10019722., 28655132.,  6492193., 28282055.],
       [ 5539008., 15835798.,  3590010., 15629916.],
       ...,
       [ 8927400., 25562400.,  5777706., 25227022.],
       [ 8613344., 24658051.,  5575906., 24335960.],
       [10693696., 30614463.,  6922140., 30213442.]])

Теперь признаки ничего не говорят о пользователях. Обучим линейную регрессию на измененных данных и посмотрим на ее качество.

In [20]:
model = LinearR(cip_vector, y)
model.score

0.42494550286671484

Метрика **R2** осталась неизменной. 

Восстановим данные и сравним их с исходными.

In [21]:
cip.decipher(cip_vector).head(10)

,Пол,Возраст,Зарплата,Члены семьи
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,2.223580e-11,46.0,38000.0,1.000000e+00
2,2.077207e-11,29.0,21000.0,-2.392672e-11
3,4.675111e-11,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,-4.578426e-11
5,1.000000e+00,43.0,41000.0,2.000000e+00
6,1.000000e+00,39.0,39700.0,2.000000e+00
7,1.000000e+00,25.0,38600.0,4.000000e+00
8,1.000000e+00,36.0,49700.0,1.000000e+00
9,1.000000e+00,32.0,51700.0,1.000000e+00


In [22]:
insurance.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


Восстановленные данные требуют небольшой обработки, т.к. нули в них записаны ввиде очень малых величин.

### Вывод

Предложенный алгоритм работает, качество модели на измененных данных не падает, а исходные данные можно восстановить.

## Общий вывод

### Краткий обзор проведенной работы

В данном проекте мы получили данные о пользователях и разработали метод их преобразования для защиты личных данных, доказав при этом, что умножение признаков на случайную обратимую никак не влияет на качество модели.

### Главные выводы

В результате выполнения проекта мы разработали метод защиты личных данных клиентов страховой компании. Его суть заключается в преобразовании признаков умножением их на случайную обратимую матрицу. Это никак не влияет на работу линейной регрессии, ее качество сохраняется. Матрицу-ключ при этом можно хранить в приватной переменной класса-шифровальщика. Исходные данные можно получить, умножив преобразованные данные на обратную ключу матрицу.